<a href="https://colab.research.google.com/github/nazanaza2970/BENTO-PACKAGING-ACTIVITY-RECOGNITION-CHALLENGE/blob/naza/Bento_Quaternion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import glob
from tqdm.notebook import trange, tqdm
from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.model_selection import train_test_split, RepeatedStratifiedKFold, cross_val_score

In [ ]:
# def segmentation(df, overlap_rate, time_window):
    
#     # make a list for segment window and its label
#     seg_data = []

#     #convert overlap rate to step for sliding window
#     overlap = int((1 - overlap_rate)*time_window)
    
#     # interpolate
#     df = df.interpolate().ffill().fillna(0)
#     #segment
#     for i in range(0, len(df)-time_window+1, overlap):
#         seg_data.append(df.loc[i:i+time_window-1, :].copy().reset_index(drop=True))
        
#     return seg_data

In [ ]:
# data = pd.DataFrame(columns = ['X1', 'Y1', 'Z1', 'X2', 'Y2', 'Z2', 'X3', 'Y3', 'Z3', 'X4', 'Y4', 'Z4',
#        'X5', 'Y5', 'Z5', 'X6', 'Y6', 'Z6', 'X7', 'Y7', 'Z7', 'X8', 'Y8', 'Z8',
#        'X9', 'Y9', 'Z9', 'X10', 'Y10', 'Z10', 'X11', 'Y11', 'Z11', 'X12',
#        'Y12', 'Z12', 'X13', 'Y13', 'Z13', 'subject_id', 'activity'])

In [ ]:
# for root, dirs, files in os.walk("/content/drive/MyDrive/Competitions/2021/Bento 2021/Data/TrainData"):
#      for file in tqdm(files):
#         tempdf = pd.read_csv(os.path.join(root,file))
#         data = data.append(segmentation(tempdf, 0.5, 500))

In [6]:
# data = data.reset_index(drop = True)
# data.to_pickle("/content/drive/MyDrive/Competitions/2021/Bento 2021/Data/windowed_500",compression = "xz")
data = pd.read_pickle("/content/drive/MyDrive/Competitions/2021/Bento 2021/Data/windowed_500_xz",compression = "xz")

In [16]:
vectors = []
# vectors.append(list(data["X1"]-data["X4"]))
# vectors.append(list(data["Y1"]-data["Y4"]))
# vectors.append(list(data["Z1"]-data["Z4"]))

# vectors.append(list(data["X1"]-data["X8"]))
# vectors.append(list(data["Y1"]-data["Y8"]))
# vectors.append(list(data["Z1"]-data["Z8"]))

# vectors.append(list(data["X8"]-data["X10"]))
# vectors.append(list(data["Y8"]-data["Y10"]))
# vectors.append(list(data["Z8"]-data["Z10"]))

vectors.append(list(data["X4"]-data["X7"]))
vectors.append(list(data["Y4"]-data["Y7"]))
vectors.append(list(data["Z4"]-data["Z7"]))

vectors.append(list(data["X13"]-data["X9"]))
vectors.append(list(data["Y13"]-data["Y9"]))
vectors.append(list(data["Z13"]-data["Z9"]))

vectors.append(list(data["X13"]-data["X6"]))
vectors.append(list(data["Y13"]-data["Y6"]))
vectors.append(list(data["Z13"]-data["Z6"]))

# vectors.append(list(data["X13"]-data["X10"]))
# vectors.append(list(data["Y13"]-data["Y10"]))
# vectors.append(list(data["Z13"]-data["Z10"]))

# vectors.append(list(data["X13"]-data["X7"]))
# vectors.append(list(data["Y13"]-data["Y7"]))
# vectors.append(list(data["Z13"]-data["Z7"]))

# vectors.append(list(data["X13"]-data["X3"]))
# vectors.append(list(data["Y13"]-data["Y3"]))
# vectors.append(list(data["Z13"]-data["Z3"]))

# vectors.append(list(data["X13"]-data["X1"]))
# vectors.append(list(data["Y13"]-data["Y1"]))
# vectors.append(list(data["Z13"]-data["Z1"]))

# vectors.append(list(data["X10"]-data["X7"]))
# vectors.append(list(data["Y10"]-data["Y7"]))
# vectors.append(list(data["Z10"]-data["Z7"]))

# vectors.append(list(data["X12"]-data["X10"]))
# vectors.append(list(data["Y12"]-data["Y10"]))
# vectors.append(list(data["Z12"]-data["Z10"]))

# vectors.append(list(data["X11"]-data["X7"]))
# vectors.append(list(data["Y11"]-data["Y7"]))
# vectors.append(list(data["Z11"]-data["Z7"]))

# vectors.append(list(data["X10"]-data["X1"]))
# vectors.append(list(data["Y10"]-data["Y1"]))
# vectors.append(list(data["Z10"]-data["Z1"]))

# vectors.append(list(data["X7"]-data["X1"]))
# vectors.append(list(data["Y7"]-data["Y1"]))
# vectors.append(list(data["Z7"]-data["Z1"]))

In [17]:
vector = np.transpose(np.array(vectors))

In [9]:
vector[:,0:3].shape

(1786500, 3)

In [10]:
def normalize(v):
    norm = np.linalg.norm(v)
    if norm == 0: 
       return v
    return v / norm

In [11]:
# def diffVectors(v1, v2):
#     """ Get rotation Quaternion between 2 vectors """
#     v1 = normalize(v1)
#     v2 = normalize(v2)
#     v = v1+v2
#     v = normalize(v)
#     angle = np.dot(v1,v2)
#     axis = np.cross(v1,v2)
#     # print(axis)
#     # print(angle)
#     return Quaternion( angle, *axis )

In [12]:
def diffVectors2(v1, v2):
    """ Get rotation Quaternion between 2 vectors """
    v1 = normalize(v1)
    v2 = normalize(v2)
    v = v1+v2
    v = normalize(v)
    angle = np.dot(v1,v2)
    axis = np.cross(v1,v2)
    # print(axis)
    # print(angle)
    return angle, axis[0],axis[1],axis[2] 

In [ ]:
quaternions = []
for i in tqdm(range(0,9,3)):
    a = vector[:,i:i+3]
    quats = []
    for i in tqdm(range(0,len(a)-1)):
        quats.append(np.array(diffVectors2(a[i],a[i+1])))
    quaternions.append(quats)

In [ ]:
# joints = ["fhls","fhrs","lslw","rsrw","vsle","vsre","vslw","vsrw"]
# # "vsrh","vsfh","lwrw","lalw","rarw","lwfh","rwfh"]

In [ ]:
pd.DataFrame(quaternions).to_pickle("/content/drive/MyDrive/Competitions/2021/Bento 2021/Data/quats2_xz.pkl",compression = "xz")

In [15]:
# pd.read_pickle("/content/drive/MyDrive/Competitions/2021/Bento 2021/Data/quats2_xz.pkl",compression = "xz")

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,1786459,1786460,1786461,1786462,1786463,1786464,1786465,1786466,1786467,1786468,1786469,1786470,1786471,1786472,1786473,1786474,1786475,1786476,1786477,1786478,1786479,1786480,1786481,1786482,1786483,1786484,1786485,1786486,1786487,1786488,1786489,1786490,1786491,1786492,1786493,1786494,1786495,1786496,1786497,1786498
0,"[0.999999999146234, 3.583202274221042e-05, -1....","[0.9999999971694722, -6.215764882644415e-05, 1...","[0.9999999995555084, -1.7518094046153898e-05, ...","[0.9999999988007271, -3.415210318541417e-05, -...","[0.9999999987169191, 4.4257111290135764e-05, 2...","[0.9999999973458423, 4.519021097748599e-05, -3...","[0.9999999983682537, -2.8356229068104977e-05, ...","[0.9999999727245977, 9.907523676266905e-05, -0...","[0.9999999874594021, 0.00014176066686144573, 2...","[0.999999977222672, 6.316936518896421e-05, -0....","[0.9999999864446656, 0.0001222175281686555, -5...","[0.9999999844289005, 8.741475830315037e-05, -0...","[0.9999999824341083, -7.273308830046954e-06, -...","[0.9999999898221116, 8.007404728310741e-05, -8...","[0.9999999973588216, 1.1582550193067753e-05, -...","[0.9999999989789444, 4.0297188836802e-05, 4.54...","[0.9999999969747098, 1.24693919394403e-05, -6....","[0.999999999833335, 1.4222720238821207e-05, -4...","[0.9999999938319071, 9.597438330505303e-05, 4....","[0.9999999933766331, -2.2100312985162684e-05, ...","[0.9999999927640836, 0.00010761363136047031, 1...","[0.999999993971019, -4.184334476864393e-05, 7....","[0.9999999994923645, 1.1802987088915806e-05, 2...","[0.9999999949481362, 1.81586217321561e-05, 9.1...","[0.9999999971538032, -6.445200500831971e-05, 5...","[0.9999999814514305, -9.381956055765084e-06, 0...","[0.9999999946182911, -9.08618396733285e-05, -3...","[0.9999999979120267, -3.847989847077793e-05, 3...","[0.9999999970049663, 4.3952673862035496e-05, 6...","[0.9999999922113825, -0.0001116637644401064, -...","[0.9999999977148789, -2.4056942483474675e-05, ...","[0.9999999983996237, -4.813097869510452e-05, 5...","[0.9999999972870778, -1.709440579616972e-05, 5...","[0.9999999945717012, -9.082441489416393e-05, 1...","[0.9999999965377077, -6.273626772496099e-05, 2...","[0.9999999980544735, 5.521813872955139e-05, 3....","[0.9999999961159496, -1.348995435468403e-05, 7...","[0.9999999990551585, -3.810128222131448e-05, -...","[0.9999999947048244, 7.555213017446105e-05, 6....","[0.999999996513212, 1.3485851294237072e-05, -6...",...,"[0.9999995256620879, 0.0007478192943220496, 0....","[0.9999997340201173, 0.000591951391651524, -2....","[0.9999997521197895, 0.0005616111614269448, 5....","[0.9999998720545749, 0.00040142998168776756, 4...","[0.9999999299688584, 0.00028954052240592887, 6...","[0.9999999747951109, 0.00016696307817917755, 6...","[0.9999999597006503, 0.0001416974643105029, 0....","[0.9999999947967619, 2.9864704112370077e-05, 8...","[0.999999977751213, -0.00014004896977867776, 0...","[0.9999999509856696, -0.0002484499087351266, 0...","[0.999999987262756, -0.00010853664502108451, 0...","[0.9999999376377895, -0.000246701134695132, 0....","[0.9999998268518834, -0.0003673723881729152, 0...","[0.9999997858507674, -0.0004951523976085004, 0...","[0.999999733373208, -0.0004863904778275585, 0....","[0.9999997544324029, -0.0005073913290032372, 0...","[0.9999997406667059, -0.0005126838425580371, 0...","[0.9999997101628303, -0.00048725387404457976, ...","[0.9999997816001458, -0.0004050530825079335, 0...","[0.9999997161470371, -0.0004715290055655186, 0...","[0.9999996613926608, -0.0004780108920311721, 0...","[0.9999997542746353, -0.000418250247667562, 0....","[0.9999997475548872, -0.0004324880710299961, 0...","[0.9999998313656449, -0.00036480260487378735, ...","[0.9999999069297029, -0.00022969219532645302, ...","[0.9999999613725644, -0.00020682152687609423, ...","[0.9999999914850762, -6.346377944030368e-05, 0...","[0.9999999526865628, 4.40742410493522e-05, 0.0...","[0.999999989006124, -7

In [ ]:
|